In [1]:
import re
import pandas as pd

In [39]:
###
# Get the courses from a file, into a dict.
###

courses = {}
fileRead = '' ## DEBUG

with open('./pages/HIS-2019-fall.txt', 'r') as course_file:
    # Read course file into one big string
    data = course_file.read()
    
    fileRead = data ## DEBUG
    
    # Break down to courses
    data = data.split('Collapse section ')
    
    # Remove erroneous space items
    del data[0]
    
    # Key the courses into the `courses` object
    for course in data:
        courses[course[0:8]] = course

In [40]:
def handle_section(section):
    sectionCode = section[0].split('-')[0]
    
    sectionData = {
        'admin': {},
        'days': []
    }
    
    ## Handling admin stuff
    sectionData['admin']['code'] = sectionCode
    sectionData['admin']['type'] = section[0].split('-')[1].split('\n')[0]
    sectionData['admin']['duration'] = section[0].split('-')[1].split('\n')[1]
    sectionData['admin']['isOpen'] = 'Open' in section[5]
    
    ## Handling days
    
    ### Helper function to extract datetime details
    def handle_section_datetime(datetime):
        return {
            'weekday': datetime[:2],
            'times': {
                'start': datetime[3:].split(' - ')[0],
                'end': datetime[3:].split(' - ')[1]
            }
        }

    ### Extract the relevant details in a very hacky way :)
    def mini_day_extractor(sectionToHandle, index):
        return {
            **handle_section_datetime(sectionToHandle[1].split('\n')[index]),
            'location': sectionToHandle[2].split('\n')[index],
            'prof': sectionToHandle[3].split('\n')[index],
            'dates': {
                'start': sectionToHandle[4].split('\n')[index].split(' - ')[0],
                'end': sectionToHandle[4].split('\n')[index].split(' - ')[1]
            }
        }
    
    ### Given the number of days (counted by the number of `\n` in the datetime string...), extract them
    for index in range(0, section[1].count('\n') + 1):
        sectionData['days'].append(mini_day_extractor(section, index))
            
    return {sectionCode: sectionData}

In [41]:
###
# Get the sections from each course, adding as a list to the dict item.
###

for courseCode, courseStr in courses.items():
    sections = {}
    
    if ' \n\t\t\n\t\n\t\t\n\t\n\t\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n' in courseStr:
        sectionContainer = courseStr.split(' \n\t\t\n\t\n\t\t\n\t\n\t\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')[1]
    else:
        sectionContainer = courseStr.split(' \n\t\t\n\t\n\t\t\n\t\n\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')[1]
    
    sectionData = sectionContainer.split('\n\t\n\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')
    
    for section in sectionData:
        sectionInfo = handle_section(section.split('\n\t\n'))
        
        sections.update(sectionInfo)
        
    courses[courseCode] = sections

In [42]:
## Clean badly processed courses :)
coursesToDelete = {}

for courseCode, courseObj in courses.items():
    if isinstance(courseObj, str):
        coursesToDelete[courseCode] = courseObj
        
for course in coursesToDelete:
    courses.pop(course, None)
        
print(str(len(coursesToDelete)) + " courses removed due to badly formed data.")

0 courses removed due to badly formed data.


In [43]:
courses

{'HIS 1101': {'A00': {'admin': {'code': 'A00',
    'duration': 'FullSess.',
    'isOpen': True,
    'type': 'LEC'},
   'days': [{'dates': {'end': '2019-12-03', 'start': '2019-09-04'},
     'location': '161 Louis Pasteur (CBY) C03',
     'prof': 'Damien-Claude Bélanger',
     'times': {'end': '14:20', 'start': '13:00'},
     'weekday': 'We'},
    {'dates': {'end': '2019-12-03', 'start': '2019-09-04'},
     'location': '550 Cumberland (TBT) 333',
     'prof': 'Damien-Claude Bélanger',
     'times': {'end': '12:50', 'start': '11:30'},
     'weekday': 'Fr'}]}},
 'HIS 1110': {'A00': {'admin': {'code': 'A00',
    'duration': 'FullSess.',
    'isOpen': True,
    'type': 'LEC'},
   'days': [{'dates': {'end': '2019-12-03', 'start': '2019-09-04'},
     'location': '125 University (MNT) 202',
     'prof': 'Eric Allina',
     'times': {'end': '12:50', 'start': '11:30'},
     'weekday': 'Mo'},
    {'dates': {'end': '2019-12-03', 'start': '2019-09-04'},
     'location': '125 University (MNT) 202',
 

In [69]:
df = pd.DataFrame.from_dict(courses, orient='index')
df = pd.DataFrame(df.stack())

df

0
HIS 1101 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 1110 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 1111 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         WB00  {'admin': {'code': 'WB00', 'type': 'LEC', 'dur...
HIS 1500 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 1501 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 1520 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2100 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
         B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
HIS 2101 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2103 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2105 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2129 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
HIS 2152 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2160 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2171 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2336 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2341 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2362 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2500 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2501 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2551 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
HIS 2552 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2576 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2597 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2707 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2735 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 2764 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3105 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3110 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
...                                                          ...
HIS 3154 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3301 B00   {'admin': {'code': 'B00', 'type': 'LEC', 'dura...
         C00   {'admin': {'code': 'C00', 'type': 'LEC', 'dura...
HIS 3315 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3355 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3397 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3550 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3728 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 3765 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4100 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
         B00   {'admin': {'code': 'B00', 'type': 'SEM', 'dura...
HIS 4135 B00   {'admin': {'code': 'B00', 'type': 'SEM', 'dura...
         C00   {'admin': {'code': 'C00', 'type': 'SEM', 'dura...
HIS 4150 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4192 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4301 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4361 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4362 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4380 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4397 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4535 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4551 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 4762 A00   {'admin': {'code': 'A00', 'type': 'LEC', 'dura...
HIS 4770 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 5103 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 5503 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 6332 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 7338 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 7738 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...
HIS 8900 A00   {'admin': {'code': 'A00', 'type': 'SEM', 'dura...

[63 rows x 1 columns]

In [70]:
df = df.reset_index()
df.columns = ['course', 'code', 'data']
df

,course,code,data
0,HIS 1101,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura..."
1,HIS 1110,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura..."
2,HIS 1111,B00,"{'admin': {'code': 'B00', 'type': 'LEC', 'dura..."
3,HIS 1111,WB00,"{'admin': {'code': 'WB00', 'type': 'LEC', 'dur..."
4,HIS 1500,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura..."
5,HIS 1501,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura..."
6,HIS 1520,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura..."
7,HIS 2100,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura..."
8,HIS 2100,B00,"{'admin': {'code': 'B00', 'type': 'LEC', 'dura..."
9,HIS 2101,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura..."


In [15]:
df2 = pd.io.json.json_normalize(courses)

In [71]:
df.loc['HIS 2100', 'A00']

KeyError: 'the label [HIS 2100] is not in the [index]'

In [72]:
pd.io.json.json_normalize(df['data'])

,admin.code,admin.duration,admin.isOpen,admin.type,days
0,A00,FullSess.,True,LEC,"[{'weekday': 'We', 'times': {'start': '13:00',..."
1,A00,FullSess.,True,LEC,"[{'weekday': 'Mo', 'times': {'start': '11:30',..."
2,B00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '11:30',..."
3,WB00,FullSess.,True,LEC,"[{'weekday': 'TB', 'times': {'start': ' 00:00'..."
4,A00,FullSess.,True,LEC,"[{'weekday': 'Mo', 'times': {'start': '13:00',..."
5,A00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '13:00',..."
6,A00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '11:30',..."
7,A00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '11:30',..."
8,B00,FullSess.,True,LEC,"[{'weekday': 'Mo', 'times': {'start': '08:30',..."
9,A00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '11:30',..."


In [82]:
df2 = df.join(pd.io.json.json_normalize(df['data']))

,course,code,data,admin.code,admin.duration,admin.isOpen,admin.type,days
0,HIS 1101,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura...",A00,FullSess.,True,LEC,"[{'weekday': 'We', 'times': {'start': '13:00',..."
1,HIS 1110,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura...",A00,FullSess.,True,LEC,"[{'weekday': 'Mo', 'times': {'start': '11:30',..."
2,HIS 1111,B00,"{'admin': {'code': 'B00', 'type': 'LEC', 'dura...",B00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '11:30',..."
3,HIS 1111,WB00,"{'admin': {'code': 'WB00', 'type': 'LEC', 'dur...",WB00,FullSess.,True,LEC,"[{'weekday': 'TB', 'times': {'start': ' 00:00'..."
4,HIS 1500,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura...",A00,FullSess.,True,LEC,"[{'weekday': 'Mo', 'times': {'start': '13:00',..."
5,HIS 1501,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura...",A00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '13:00',..."
6,HIS 1520,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura...",A00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '11:30',..."
7,HIS 2100,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura...",A00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '11:30',..."
8,HIS 2100,B00,"{'admin': {'code': 'B00', 'type': 'LEC', 'dura...",B00,FullSess.,True,LEC,"[{'weekday': 'Mo', 'times': {'start': '08:30',..."
9,HIS 2101,A00,"{'admin': {'code': 'A00', 'type': 'LEC', 'dura...",A00,FullSess.,True,LEC,"[{'weekday': 'Tu', 'times': {'start': '11:30',..."


In [17]:
df.columns.str.split('.')

Index([    ['HIS 1100', 'A00', 'admin', 'code'],
       ['HIS 1100', 'A00', 'admin', 'duration'],
         ['HIS 1100', 'A00', 'admin', 'isOpen'],
           ['HIS 1100', 'A00', 'admin', 'type'],
                    ['HIS 1100', 'A00', 'days'],
           ['HIS 1110', 'B00', 'admin', 'code'],
       ['HIS 1110', 'B00', 'admin', 'duration'],
         ['HIS 1110', 'B00', 'admin', 'isOpen'],
           ['HIS 1110', 'B00', 'admin', 'type'],
                    ['HIS 1110', 'B00', 'days'],
       ...
           ['HIS 6103', 'A00', 'admin', 'code'],
       ['HIS 6103', 'A00', 'admin', 'duration'],
         ['HIS 6103', 'A00', 'admin', 'isOpen'],
           ['HIS 6103', 'A00', 'admin', 'type'],
                    ['HIS 6103', 'A00', 'days'],
           ['HIS 7330', 'A00', 'admin', 'code'],
       ['HIS 7330', 'A00', 'admin', 'duration'],
         ['HIS 7330', 'A00', 'admin', 'isOpen'],
           ['HIS 7330', 'A00', 'admin', 'type'],
                    ['HIS 7330', 'A00', 'days']],
      dt

In [21]:
df.T[['Course', 'Section']] = df[0].split('.')

KeyError: 0